# Desafio: Algoritmo de IA

Usaremos o dataset encontrado em https://www.kaggle.com/wenruliu/adult-income-dataset.

Para analisar o dataset, utilizaremos o método de classificação chamado de *K Nearest Neighbors*.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
adult = pd.read_csv('adult.csv')

In [3]:
import sklearn
##Algoritmo em questão
from sklearn.neighbors import KNeighborsClassifier
##Para normalização das features
from sklearn.preprocessing import StandardScaler
##Para separar o dataset a ser treinado e no qual ser testado
from sklearn.model_selection import train_test_split
##Para analisar efetividade do algoritmo
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

Para podermos abordar o problema de classificação, transformamos nosso *output* em valores inteiros (no caso, $0$ e $1$): 

In [4]:
adult['income']=adult['income'].replace("<=50K",0)
adult['income']=adult['income'].replace(">50K",1)

Construímos nosso dataset $X$ como as *features* que não são "income", e nosso output como a coluna "income"; para simplicidade, vou somente analisar as colunas numéricas (note-se que eventuais resultados serão afetados por essa escolha):

In [5]:
columns=['workclass','education','marital-status','occupation','relationship','race','gender','native-country']
adult = adult.drop(columns,axis=1)
X=adult.iloc[:,0:-1] ##Vai até, mas não inclui, coluna "income"
y=adult.iloc[:,-1] ##Somente última coluna, "income"

Vemos os respectivos *heads*, só para um *sanity check*:

In [6]:
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,25,226802,7,0,0,40
1,38,89814,9,0,0,50
2,28,336951,12,0,0,40
3,44,160323,10,7688,0,40
4,18,103497,10,0,0,30


In [7]:
y.head()

0    0
1    0
2    1
3    1
4    0
Name: income, dtype: int64

Reservando 20% dos dados para teste, e 80% para treinar o algoritmo:

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Para uma análise mais eficiente, normalizamos as *features* de nossos dados em $X_{train}$ e $X_{test}$:

In [9]:
normalize=StandardScaler()
X_train = normalize.fit_transform(X_train)
X_test=normalize.fit_transform(X_test)

Agora, para o algoritmo em si:

In [10]:
##A raiz do tamanho do dataset é uma primeira estimativa de quantas "classes" ele tem
K=int(np.sqrt(len(X_train)))
##Instanciando o algoritmo, com métrica/distância Euclidiana
knn = KNeighborsClassifier(n_neighbors=K,metric='euclidean')
##Treinando o algoritmo
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=197, p=2,
                     weights='uniform')

Com o algoritmo treinado, podemos prever os valores que $X_{test}$ tomariam, e comparar com seus valores $y_{test}$:

In [11]:
y_predict=knn.predict(X_test)

Uma análise ingênua, com quantas vezes o algoritmo acertou, sobre todas suas tentativas:

In [12]:
accuracy_score(y_test,y_predict)

0.8209642747466476

Analisando o teste F (que é uma medida do *trade-off* entre precisão e *recall*):

In [13]:
f1_score(y_test,y_predict)

0.5175172413793103

Os resultados muito provavelmente foram afetados pela ausência dos dados categóricos (por exemplo, com a questão racial e de gênero, de diferenças salariais), assim como *outliers* (por exemplo, em *hours-per-week* ou *capital-gain*).